In [13]:
import requests
from bs4 import BeautifulSoup
import csv

base_url = "https://www.sports-reference.com/cbb/schools/"

response = requests.get(base_url)

soup = BeautifulSoup(response.content, "html.parser")

table = soup.find("table", {"id": "NCAAM_schools"})

rows = []

# Extract text content from <td> elements with data-stat="school_name"
for row in table.find_all("td", {"data-stat": "school_name"}):
    rows.append(row)

schools = []
for row in rows:
    schools.append(row.find("a"))

# Attempt to get the href attribute from each school
links = []
for school in schools:
    try:
        # Directly try to get the href attribute from the <td> element
        links.append(school.get("href"))
        
    except Exception as e:
        print(e)

modified_links = []

# Add "2023-gamelogs-advanced.html" to the end of each link
for link in links:
    modified_link = "https://www.sports-reference.com" + link + "2023-gamelogs-advanced.html"
    modified_links.append(modified_link)
    print(modified_link)




AttributeError: 'NoneType' object has no attribute 'find_all'

In [ ]:
import time

for url in modified_links:
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check for HTTP errors

        soup = BeautifulSoup(response.content, "html.parser")

        # Find the target table
        table = soup.find("table", {"id": "sgl-advanced"})

        # Check if the table is found
        if table is None:
            print(f"No table found on the page for URL: {url}")
            continue

        # Extract header row
        headers = [th.text.strip() for th in table.find_all("tr")[1].find_all("th")]

        # Extract data from remaining rows
        data = []
        for row in table.find_all("tr")[1:]:  # Skip header row
            cells = row.find_all("td")
            data_row = [cell.text.strip() for cell in cells]
            data.append(data_row)

        # Get the team name from the url
        team_name = url.split("/")[-3]

        print(team_name)

        file = f"./team_stats/{team_name}_adv_2023.csv"

        with open(file, "w") as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(headers)
            writer.writerows(data)

        # Introduce a delay to avoid rate limiting
        time.sleep(3)  # You can adjust the delay time as needed

    except requests.exceptions.RequestException as e:
        print(f"Error making the request for URL {url}: {e}")
        continue

    except Exception as e:
        print(f"Error processing URL {url}: {e}")
        continue